<a href="https://colab.research.google.com/github/KangSangKyu/BOOKSTUDY_Python-Machine-Learning-Perfect-Guide/blob/main/Example_5_Bike_demand_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bike demand prediction

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

bike_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/train_1204.csv')

print(bike_df.shape)
bike_df.head()

In [ ]:
bike_df.info()

In [ ]:
# 문자열 datetime 타입으로 변경 

bike_df['datetime'] = bike_df.datetime.apply(pd.to_datetime)

# datetime 타입에서 년, 월, 일, 시간 추출

bike_df['year'] = bike_df.datetime.apply(lambda x : x.year)
bike_df['month'] = bike_df.datetime.apply(lambda x : x.month)
bike_df['day'] = bike_df.datetime.apply(lambda x : x.day)
bike_df['hour'] = bike_df.datetime.apply(lambda x : x.hour)

bike_df.head()

In [ ]:
# 칼럼 삭제 

drop_columns = ['datetime', 'casual', 'registered']

bike_df.drop(drop_columns, axis=1, inplace=True)
bike_df.head()

In [ ]:
# RMSLE, RMSE, MSE 를 구하는 함수 

from sklearn.metrics import mean_squared_error, mean_absolute_error

# log 값 반환 시 NaN 등의 이슈로 log()가 아닌 log1p()를 이용해 RMSE 계산 

def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle = np.sqrt(np.mean(squared_error))
    
    return rmsle

# 사이킷런의 mean_square_error()를 이용해 RMSE 계산 

def rmse(y, pred):
    return np.sqrt(mean_squared_error(y, pred))

# MSE, RMSE, RMSLE 를 모두 계산 

def evaluate_regr(y, pred):
    rmsle_val = rmsle(y, pred)
    rmse_val = rmse(y, pred)
    
    # MSE는 사이킷런의 mean_absolute_error()로 계산
    mse_val = mean_absolute_error(y, pred)
    
    print('RMSLE: {0:.3f}, RMSE: {1:3f}, MSE: {2:3f}'.format(rmsle_val, rmse_val, mse_val))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso

y_target = bike_df['count']
X_features = bike_df.drop(['count'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3, random_state=0)

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

evaluate_regr(y_test, pred)

In [ ]:
def get_top_error_data(y_test, pred, n_tops=5):
    
    # 데이터프레임의 칼럼으로 실제 대여 횟수(count)와 예측값을 서로 비교할 수 있도록 생성 
    result_df = pd.DataFrame(y_test.values, columns=['real_count'])
    result_df['predicted_count'] = np.round(pred)
    result_df['diff'] = np.abs(result_df['real_count'] - result_df['predicted_count'])
    
    # 예측값과 실제 값이 가장 큰 데이터 순으로 출력 
    print(result_df.sort_values('diff', ascending=False)[:n_tops])
    
get_top_error_data(y_test, pred, n_tops=5)

In [ ]:
y_target.hist()

In [ ]:
y_log_transform = np.log1p(y_target)
y_log_transform.hist()

In [ ]:
from sklearn.linear_model import LinearRegression

# 타깃 칼럼인 count 값을 log1p로 로그 변환 

y_target_log = np.log1p(y_target)

# 로그 변환된 y_target_log를 반영해 학습 / 테스트 세트 분할 

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target_log, test_size=0.3, random_state=0)

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

# 테스트 데이터 세트의 Target 값은 로그 변환됐으므로 다시 expm1 을 이용해 원래 스케일로 변환 

y_test_exp = np.expm1(y_test)

# 예측값 역시 로그 변환된 타깃 기반으로 학습돼 예측했으므로 다시 expm1 으로 스케일 변환 

pred_exp = np.expm1(pred)

evaluate_regr(y_test_exp, pred_exp)

In [ ]:
# 각 회귀 계수의 값 시각화 하기 

coef = pd.Series(lr_reg.coef_, index=X_features.columns)
coef_sort = coef.sort_values(ascending=False)
sns.barplot(x=coef_sort.values, y=coef_sort.index)

In [ ]:
# 'year', 'month', 'hour', 'season', 'wather' 피처를 원-핫 인코딩

X_features_ohe = pd.get_dummies(X_features, columns=['year', 'month', 'hour', 'holiday', 'workingday', 'season', 'weather'])

In [ ]:
# 원-핫 인코딩이 적용된 피처 데이터 세트 기반으로 학습 / 예측 데이터 분할 

X_train, X_test, y_train, y_test = train_test_split(X_features_ohe, y_target_log, test_size=0.3, random_state=0)

# 모델과 학습 / 테스트 데이터 세트를 입력하면 성능 평가 수치를 반환 

def get_model_predict(model, X_train, X_test, y_train, y_test, is_expm1=False):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    if is_expm1 :
        y_test = np.expm1(y_test)
        pred = np.expm1(pred)
    print('###', model.__class__.__name__, '###')
    evaluate_regr(y_test, pred)
    
# end of function get_model_predict

# 모델별로 평가 수행 

lr_reg = LinearRegression()
ridge_reg = Ridge(alpha=10)
lasso_reg = Lasso(alpha=0.01)

for model in [lr_reg, ridge_reg, lasso_reg]:
    get_model_predict(model, X_train, X_test, y_train, y_test, is_expm1=True)

In [ ]:
coef = pd.Series(lr_reg.coef_, index=X_features_ohe.columns)
coef_sort = coef.sort_values(ascending=False)[:10]
sns.barplot(x=coef_sort.values, y=coef_sort.index)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# 랜덤 포레스트 GBM, XGBoost, LightGBM 모델별로 평가 수행 

rf_reg = RandomForestRegressor(n_estimators=500)
gbm_reg = GradientBoostingRegressor(n_estimators=500)
xgb_reg = XGBRegressor(n_estimators=500)
lgbm_reg = LGBMRegressor(n_estimators=500)

for model in[rf_reg, gbm_reg, xgb_reg, lgbm_reg]:
    get_model_predict(model, X_train, X_test, y_train, y_test, is_expm1=True)